In [21]:
%pip install pinecone-client

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [22]:
import pandas as pd

df = pd.read_parquet('data/embeddings.parquet')
df.head()

,classification,id,filename,page_num,text,context,n_tokens,embeddings
0,Rubbish,128500,128500/10.pdf,1,title check tenure: freehold do we have access...,title check tenure: freehold do we have access...,299,"[-0.0032051908783614635, -0.000574587029404938..."
1,Rubbish,128500,128500/10.pdf,2,notes (portal) clean title set up 16/11 awaiti...,notes (portal) clean title set up 16/11 awaiti...,14,"[-0.019090935587882996, 0.001389871584251523, ..."
2,Rubbish,128500,128500/10.pdf,1,title check tenure: freehold do we have access...,title check tenure: freehold do we have access...,299,"[-0.0032051908783614635, -0.000574587029404938..."
3,Rubbish,128500,128500/11.pdf,1,these are the notes referred to on the followi...,these are the notes referred to on the followi...,224,"[-0.02158627286553383, -0.007079085800796747, ..."
4,Rubbish,128500,128500/11.pdf,2,this official copy is incomplete without the p...,this official copy is incomplete without the p...,11,"[-0.013049332424998283, 0.00461492408066988, -..."


In [23]:
from sys import getsizeof

too_big = []

for text in df['context'].tolist():
    if getsizeof(text) > 10000:
        too_big.append((text, getsizeof(text)))

print(f"{len(too_big)} / {len(df)} records are too big")

0 / 1537 records are too big


In [24]:
df['context_id'] = [str(i) for i in range(len(df))]
df.head()

,classification,id,filename,page_num,text,context,n_tokens,embeddings,context_id
0,Rubbish,128500,128500/10.pdf,1,title check tenure: freehold do we have access...,title check tenure: freehold do we have access...,299,"[-0.0032051908783614635, -0.000574587029404938...",0
1,Rubbish,128500,128500/10.pdf,2,notes (portal) clean title set up 16/11 awaiti...,notes (portal) clean title set up 16/11 awaiti...,14,"[-0.019090935587882996, 0.001389871584251523, ...",1
2,Rubbish,128500,128500/10.pdf,1,title check tenure: freehold do we have access...,title check tenure: freehold do we have access...,299,"[-0.0032051908783614635, -0.000574587029404938...",2
3,Rubbish,128500,128500/11.pdf,1,these are the notes referred to on the followi...,these are the notes referred to on the followi...,224,"[-0.02158627286553383, -0.007079085800796747, ...",3
4,Rubbish,128500,128500/11.pdf,2,this official copy is incomplete without the p...,this official copy is incomplete without the p...,11,"[-0.013049332424998283, 0.00461492408066988, -...",4


In [25]:
import pinecone

pinecone.init(
    api_key='3e891c00-e05a-424d-8f9d-3bce6133520a',  # app.pinecone.io
    environment='us-west1-gcp'
)

index_name = 'eqr-doc-classifier'

if not index_name in pinecone.list_indexes():
    pinecone.create_index(
        index_name, dimension=len(df['embeddings'].tolist()[0]),
        metric='cosine'
    )

index = pinecone.Index(index_name)

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=UTF-8', 'date': 'Wed, 04 Jan 2023 19:24:29 GMT', 'content-length': '123', 'x-envoy-upstream-service-time': '0', 'server': 'envoy'})
HTTP response body: Index name must consist of lower case alphanumeric characters or '-', and must start and end with an alphanumeric character


In [13]:
from tqdm.auto import tqdm

batch_size = 32

for i in tqdm(range(0, len(df), batch_size)):
    i_end = min(i+batch_size, len(df))
    df_slice = df.iloc[i:i_end]
    to_upsert = [
        (
            row['id'],
            row['embeddings'].tolist(),
            {
                'docs': row['Context'],
                'file_type': row['FileType'],
                'origin': row['Origin'],
                'page_no': row['PageNo'],
                'n_tokens': row['n_tokens']
            }
        ) for _, row in df_slice.iterrows()
    ]
    index.upsert(vectors=to_upsert)

/home/vscode/.local/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 18/18 [00:10<00:00,  1.70it/s]


In [14]:
mappings = {row['id']: row['text'] for _, row in df[['id', 'text']].iterrows()}

In [15]:
import json

with open('data/mapping.json', 'w') as fp:
    json.dump(mappings, fp)